In [1]:
using Pkg
Pkg.add("Distributed") ;
Pkg.add("LinearAlgebra") ;
Pkg.add("Random") ;
using Distributed, Random, LinearAlgebra ;

    Updating registry at `~/.julia/registries/General`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.6/Project.toml`
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.6/Project.toml`
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.6/Project.toml`
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`


In [2]:
length(Sys.cpu_info())
addprocs(3)

3-element Vector{Int64}:
 2
 3
 4

In [3]:
nworkers()

3

In [17]:
@everywhere using  Random, LinearAlgebra, Distributed

In [5]:
# run rand(2,2) and rand(1:8, 3,4) on worker 2
r1 = remotecall(rand, 2, 2,2)
r2 = remotecall(rand, 2, 1:8,3,4)

Future(2, 1, 12, nothing)

In [6]:
# retrieve the results
"""
To treat the output as normal output we must save it as another variable first, 
r1 & r2 are still type: Future so cannot do matrix things or whatever on them 
"""
r11 = fetch(r1)
r22 = fetch(r2)

3×4 Matrix{Int64}:
 1  7  6  7
 1  6  2  7
 7  3  8  5

In [7]:
"""
Can send task to a worked without specifying the worker
"""
s1 = @spawn rand(2,2) # s1 is still a Future type
s2 = fetch(s1) # s2 is an Array

2×2 Matrix{Float64}:
 0.559097  0.179241
 0.10152   0.982152

In [8]:
"""
Can do send tasks for worker to do on the things
"""
s3 = @spawn 1 .+ fetch(s1) # no need to specify worker (Julia selects it)
r3 = @spawnat 2 1 .+ fetch(r1) # worker specified as first element


Future(2, 1, 18, nothing)

In [19]:
"""
for efficiency, remotecall_fetch is quicker than fetch(remotecall())
IS NOT WORKING 
GO F*** YOURSELF, remotecall_fetch. I HATE YOU!!!!!!
"""
# remotecall_fetch(rand, 2, 2,2)
fetch(remotecall(rand, 2, 2,2))

2×2 Matrix{Float64}:
 0.0281693  0.632558
 0.540143   0.211267

In [10]:
function eig_sum(A) # computes the sum of the eigenvalues
    autoVal = eigvals(A)
    return sum(autoVal)
end

s1 = @spawnat 1 eig_sum(rand(2,2))
fetch(s1)
"""
the function is definied on worked 1 (the parent/master node) but not any of the workers


s2 = @spawnat 2 eig_sum(rand(2,2)) 
fetch(s2) # returns an error

to define something on all workers we can use @everywhere
"""

@everywhere function eig_sum(A)
    sum(eigvals(A))
end

s2 = @spawnat 2 eig_sum(rand(2,2))
fetch(s2)

1.11180940918371

In [11]:
"""
DATA MOVEMENT                DATA MOVEMENT                  DATA MOVEMENT
DATA MOVEMENT                DATA MOVEMNET                  DATA MOVEMENT
"""

"DATA MOVEMENT                DATA MOVEMENT                  DATA MOVEMENT\nDATA MOVEMENT                DATA MOVEMNET                  DATA MOVEMENT\n"

In [12]:
"""
data movemnet idea
"""
A = rand(10,10)
A2 = @spawn A^2 # data is moved, locally -> [some] worker
A22 = @spawn rand(10,10)^2 # less data movement, as matrix created by worker

Future(2, 1, 29, nothing)

In [13]:
"""
parallel For loop, counting the number of 1's from a Bernoulli random variable
"""
n = 200000000
nhead = @distributed for i=1:n
                Int(rand(Bool)) 
end

Task (runnable) @0x000000011d6b3420

In [14]:
n

200000000

In [15]:
"""
estimating pi via monte carlo methods
"""
pi = 0.0
pi = @distributed (+) for i=1:n
    Int(rand()^2 + rand()^2 <= 1);
end
pi /= n/4

3.14165024

In [16]:
a = 0;
@time for i=1:20000
    a += Int(rand(Bool));
end

@time @distributed (+) for i=1:20000
    Int(rand(Bool));
end

@time sum(rand(0:1, 20000));

n = 200000000;
@time @distributed (+) for i=1:n
    Int(rand(Bool));
end


  0.000519 seconds (18.95 k allocations: 296.141 KiB)
  0.022213 seconds (48.15 k allocations: 2.725 MiB, 39.12% compilation time)
  0.000213 seconds (2 allocations: 156.328 KiB)
  0.427328 seconds (48.24 k allocations: 2.733 MiB, 1.89% compilation time)


100001086

In [ ]:
"""
DYNAMIC SCHEDULING                       DYNAMIC SCHEDULING
DYNAMIC SCHEDULING                       DYNAMIC SCHEDULING 
Often and for my stochastic simulations tasks will take varying amounts of time to complete
dynamic scheduling allows new tasks to be started as soon as the previous has been
completed, rather than waiting, idle, useless and impedent for other tasks to be finished.
"""

In [ ]:
# The arguments are: 1) a function 'f' and 2) a list with the input.
function f_pmap(f, lst)
    np = nprocs()            # Number of processes available.
    n  = length(lst)         # Number of elements to apply the function.
    results = Vector{Any}(n) # Where we will write the results. As we do not know
                             # the type (Integer, Tuple...) we write "Any"
    i = 1
    nextidx() = (idx = i; i += 1; idx) # Function to know which is the next work item.
                                       # In this case it is just an index.
    @sync begin #@sync: must complete all jobs in block
        for p = 1:np # loops through all processes (workers)
            if p != myid() || np == 1 # first worker used only if all others are busy 
                @async begin # launch several tasks simultaneaously
                    while true
                        idx = nextidx()
                        if idx > n
                            break
                        end
                        results[idx] = fetch(remotecall(f, p, lst[idx]))
                    end
                end
            end
        end
    end
    results
end